In [ ]:

   
from time import time
from PyQt5.QtWidgets import QDialog ,QApplication, QLabel,QComboBox, QPushButton, QTextEdit, QProgressBar, QTableWidget,QTableWidgetItem
from PyQt5 import uic, QtCore
import sys
import sortingAlgorithms as sAlgo
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import requests
import time





class UI(QDialog):
    
    #  sorting algorithms classes objects
    mergeObj = sAlgo.AscendingMerge()
    insertionObj = sAlgo.AscendingInsertion()
    selectionObj = sAlgo.AscendingSelection()
    quickObj = sAlgo.AscendingQuick()
    
    def __init__(self):
        super(UI,self).__init__()
        uic.loadUi("Project Ui.ui", self)
        self.show()
        self.dataTable.viewport().installEventFilter(self)
        self.dataTable.horizontalHeader().sectionClicked.connect(self.applySortingOnColumn)
        
    # function to apply sorting a particular column when clicked on the header 
    def applySortingOnColumn(self):
        rowCount  = self.dataTable.rowCount()
        column = self.dataTable.currentColumn()
        
        array = self.getColumnData(rowCount, column)
        selectedAlgorithm = self.sortingAlgoComboBox.currentText()
        
        if selectedAlgorithm == "Merge Sort":
            self.mergeObj.mergeSort(array, 0, len(array)-1)
            self.mergeObj = self.mergeObj.mergeOrder()
            
        if selectedAlgorithm == "Insertion Sort":
            self.insertionObj.insertionSort(array)
            self.insertionObj = self.insertionObj.insertionOrder()
            
        if selectedAlgorithm == "Selection Sort":
            self.selectionObj.selectionSort(array)
            self.selectionObj = self.selectionObj.selectionOrder()
            
        if selectedAlgorithm == "Quick Sort":
            self.quickObj.quickSort(array, 0, len(array)-1)
            self.quickObj = self.quickObj.quickOrder()

        if selectedAlgorithm == "Bubble Sort":
            self.bubbleObj.bubbleSort(array)
            self.bubbleObj = self.bubbleObj.bubbleOrder()

        if selectedAlgorithm == "Exchange Sort":
            self.exchangeObj.exchangeSort(array)
            self.exchangeObj = self.exchangeObj.exchangeOrder()
            
        
        self.setColumnData(rowCount, column, array)
        
    # function to get data of the clicked column to apply sorting
    def getColumnData(self, row, column):
        columnData = []
        for i in range(0, row):
            columnData.append(self.dataTable.item(i, column).text())
        return columnData
    
    # function to insert data in the column after sorting
    def setColumnData(self, row, column, array):
        for i in range(0,row):
            self.dataTable.setItem(i, column, QTableWidgetItem(str(array[i])))
            
    
    # function just to check which row or column is clicked    
    def eventFilter(self, source, event):
        if self.dataTable.selectedIndexes() != []:
            if event.type() == QtCore.QEvent.MouseButtonPress:
                if event.button() == QtCore.Qt.LeftButton:
                    row = self.dataTable.currentRow()
                    col = self.dataTable.currentColumn()
                elif event.button() == QtCore.Qt.RightButton:
                    row = self.dataTable.currentRow()
                    col = self.dataTable.currentColumn()
       
        return QtCore.QObject.event(source, event)
    
    #function for starting scraping
    def startScraping(self):
        
        urlPart1 = ""
        urlPart2 = ""
        baseUrl = "https://www.ilaan.com"
        links = []
        houseTitle = []
        houseEstate = []
        bathRoom = []
        bedRoom = []
        houseStatus = []
        housePrice = []
        houseLocation = []

        driver = webdriver.Chrome(executable_path='C:\\ChromeDriver\chromedriver.exe')
        driver.get(baseUrl)
        content = driver.page_source
        soup = BeautifulSoup(content,"lxml")
        linksCount = 0
        for div in soup.findAll('div',{'class':'w3-dropdown-content w3-bar-block shadow-sm border-custom'}):
            a = div.findAll('a')
            for link in a:
            if linksCount< 5:
                href = link.get('href')
                links.append(href)
                linksCount = linksCount + 1

        linksCount = 0
        for link in links:
            maxPage = 0
            linksCount = linksCount + 1
            driver.get(baseUrl + link)
            content = driver.page_source
            soup = BeautifulSoup(content,"lxml")
            for a in soup.findAll('a',{'class':'page-link rounded-xs color-black bg-transparent bg-theme shadow-l border-0', 'rel' : True}):
                
                if a.find('i', {'class' : 'fa fa-fast-forward'}):
                    maxPage = a.get('rel')
          
               for i in range(1, int(maxPage[0])+1):
                driver.get((baseUrl + link +  f'?tl={linksCount}&ptid=1&pgno={i}&ic=%26ic%3dLahore'))
                content = driver.page_source
                soup = BeautifulSoup(content,"lxml")
                for div in soup.findAll('div' , attrs = {'class' : 'card card-style'}):
                    titleVaribale = ""
                    eStateVaribale = ""
                    bedVaribale = ""
                    bathVaribale = ""
                    locationVaribale = ""
                    statusVaribale = ""
                    priceVaribale = ""

                for title in div.findAll('h2' , attrs = {'class' : 'font-14 font-600 opacity-70 line-height-s mt-1'}):
                    titleVaribale = title.text

                for estate in div.findAll('h6' , attrs = {'class' : 'font-11 font-500 opacity-60 mt-2 text-center'}):
                    estateVaribale = estate.text

                for location in div.findAll('p' , attrs = {'class' : 'font-11 me-1 opacity-70'}):
                    locationVaribale = location.text
            
                for room in div.findAll('span' , attrs = {'class' : 'color-white opacity-70'}):
                    if room.find('i', attrs={'class': 'fa fa-bath pe-2'}):
                        bathVaribale = int(room.text)
                        
                    if room.find('i', attrs={'class': 'fa fa-bed pe-2'}):
                        bedVaribale = int(room.text)

                    for price in div.findAll('strong' , attrs = {'class' : 'color-highlight'}):
                        priceVaribale = (price.parent).text

                for status in div.findAll('h3' , attrs = {'class' : 'font-10 font-600 mb-n1 badge bg-red-dark'}):
                    statusVaribale = status.text
                    
                if titleVaribale != "":
                    houseTitle.append(titleVaribale)
                
                 if locationVaribale != "":
                    houseLocation.append(locationVaribale)
                else:
                    houseLocation.append("")

                if bedVaribale != "":
                    bedRoom.append(bedVaribale)
                else:
                    bedRoom.append("0")

                if bathVaribale != "":
                    bathRoom.append(bathVaribale)
                else:
                    bathRoom.append("0")

                if priceVaribale != "":
                    housePrice.append(priceVaribale)
                else:
                    housePrice.append("0")

                if statusVaribale != "":
                    houseStatus.append(statusVaribale)
                else:
                    houseStatus.append("")

                if estateVaribale != "":
                    houseEstate.append(estateVaribale)
                else:
                    houseEstate.append("")
                    
                self.dataTable.setRowCount(0)
                RC = self.dataTable.rowCount()
                for row in range (RC):
                    count = self.dataTable.rowCount()
                    self.dataTable.insertRow(count)
                    self.dataTable.setItem(count, 0, QTableWidgetItem(str(houseTitle[row])))
                    self.dataTable.setItem(count, 0, QTableWidgetItem(str(houseEstate[row])))
                    self.dataTable.setItem(count, 0, QTableWidgetItem(str(bathRoom[row])))
                    self.dataTable.setItem(count, 0, QTableWidgetItem(str(bedRoom[row])))
                    self.dataTable.setItem(count, 0, QTableWidgetItem(str(houseStatus[row])))
                    self.dataTable.setItem(count, 0, QTableWidgetItem(str(housePrice[row])))
                    self.dataTable.setItem(count, 0, QTableWidgetItem(str(houseLocation[row])))
                    
                
            
                    
app = QApplication(sys.argv)
UIWindow = UI()
app.exec_()

    
    
